In [1]:
import json
import re
import os

In [2]:
def transform_text(interpreted):
    text = interpreted['text']
    if interpreted['entities']:
        add = 0
        for entity in interpreted['entities']:
            text = text[:entity['start']+add]+"["+entity['value']+"]("+entity['entity']+")"+text[entity['end']+add:]
            add += 4+len(entity['entity'])
    return text

## Intents from conversations

In [4]:
conversation_files = os.listdir("conversations")

In [5]:
intents = {}
for conversation_file in conversation_files:
    if conversation_file[-4:] == "json":
        with open("conversations/{}".format(conversation_file)) as f:
            conv = json.load(f)
            for message in conv:
                if message['from'] == "user":
                    if intents.get(message['intent']) is None:
                        intents[message['intent']] = []
                    intents[message['intent']].append(transform_text(message['interpreted']))

In [6]:
intents

{'yes_simple': ['I want to order something',
  'Yes, I want to make an order',
  'ok',
  'yes'],
 'yes_x_y': ['Yes, I want some [nachos](order_item)']}

## Intents from disambiguisations

In [7]:
intents = {}
for conversation_file in conversation_files:
    if conversation_file[-4:] == "json":
        with open("conversations/{}".format(conversation_file)) as f:
            conv = json.load(f)
            for i in range(0, len(conv)):
                message = conv[i]
                if message['from'] == "bot" and message.get("disambiguisation"):
                    if len(conv)>i+1 and conv[i+1]['interpreted']['intent']['confidence'] >= 0.9:
                        print(conv[i-1])
                        intent = conv[i+1]['interpreted']['intent']['name']
                        if intents.get(intent) is None:
                            intents[intent] = []
                        
                        intents[intent].append(transform_text(conv[i-1]['interpreted']))

{'from': 'user', 'message': 'I want to order something', 'intent': 'yes_simple', 'interpreted': {'entities': [], 'intent': {'confidence': 0.4787919819355011, 'name': 'yes_simple'}, 'intent_ranking': [{'confidence': 0.4787919819355011, 'name': 'yes_simple'}, {'confidence': 0.4017016589641571, 'name': 'yes_x_y'}, {'confidence': 0.11359098553657532, 'name': 'decline'}, {'confidence': 0.0059153735637664795, 'name': 'yes_not_sure'}], 'text': 'I want to order something'}}


In [8]:
intents

{'yes_simple': ['I want to order something']}